<a href="https://colab.research.google.com/github/gar-nov/tirocinio/blob/main/clean_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# introduzione alla MISBEHAVIOUR DETECTION nei VANETS
i VANETS ( Vehicular Ad-Hoc Networks ) sono reti in cui i nodi, ovvero veicoli, comunicano tra loro (V2V: Vehicle-to-vehicle) e con le infrastutture stradali tramite moduli wireless ad hoc attraverso protocolli di comunicazione specifici come il DSRC (Dedicated short Range Communication) o lo IEEE 802.11p, il cui obbiettivo sono migliorare la sicurezza stradale, rendere il traffico fluido e gestire la mobilità.
I veicoli comunicano tra di loro scambiandosi messaggi periodicin chiamati  Basic Safety Messages (BSM) negli Stati Uniti o i Cooperative Awarness Messages (CAM) in Europa e contengono informazioni su posizione, velocità, direzione e altri parametri del veicolo.
Nonostante i protocolli utilizzati garantiscano che i messaggi siano autentici, un veicolo malevolo può inviare messaggi con contenuti falsi e per affrontare questo problema si introduce il concetto di Misbehavior Detection che sono un insieme di tecniche che consentono di rilevare messaggi non corretti o comportamenti sospetti all'interno della rete.

Esitono due tipologie di approcci:
- Node Centric: stiamo la fiducia o reputazione del mittente basandosi sul suo comportamento storico
- Data-centric: verificano se i dati ricevuti sono attendibili

*(Fonte: Van der Heijden et al., "Survey on Misbehavior Detection in Cooperative Intelligent Transportation Systems", ACM Computing Surveys, 2016)*


In questo progetto ci concentriamo su un approccio di tipo data-centric in cui si andranno a valutare attraverso tecniche di Machine Learning se i dati ricevuti dai veicoli che sono presenti nel dataset Veremi, sono affidabili oppure rappresentano un attacco.
Il dataset utilizza lo scenario LuST, che simula realisticamente il traffico di Lussemburgo. Non si concentra su un solo tipo di traffico, ma include situazioni diverse: strade urbane, periferiche, e autostrade. Per coprire più condizioni, hanno simulato tre momenti della giornata: 3:00 di notte per traffico basso, 5:00 per traffico medio e 7:00 di mattina per traffico alto. Ogni simulazione dura 100 secondi e ci sono in totale 225 simulazioni ottenute variando la densità, la probabilità di avere attaccanti e i semi casuali.




#Obbiettivo: pulire il dataset

analizzare la struttura e il contenuto del dataset in particolare si vogliono controllare i valori nulli ed eventualmente gestirli, rimuovere la presenza di duplicati, rimuovere campi non utili.

# Importazione e installazione di librerie

importiamo le librerie necessarie per manipolare il dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install kagglehub --quiet
import kagglehub

# caricamento del dataset veremi

In [3]:
path = kagglehub.dataset_download("haider094/veremi-dataset")


100%|██████████| 3.14G/3.14G [00:31<00:00, 106MB/s]

Extracting files...


In [4]:
import os

for root, dirs, files in os.walk(path):
    for name in files:
        print(os.path.join(root, name))

/root/.cache/kagglehub/datasets/haider094/veremi-dataset/versions/1/Veremi_final_dataset.csv


In [5]:
data = pd.read_csv(f"{path}/Veremi_final_dataset.csv")

# Analisi struttura e contenuto del dataset

In [6]:
# comando per visualizzare le prime righe del contenuto
data.head()

,Unnamed: 0,type,rcvTime,pos_0,pos_1,pos_noise_0,pos_noise_1,spd_0,spd_1,spd_noise_0,...,acl_0,acl_1,acl_noise_0,acl_noise_1,hed_0,hed_1,hed_noise_0,hed_noise_1,attack,attack_type
0,0,3,27371.216284,981.098535,908.497891,3.826423,3.964059,-17.723786,-2.154895,-0.024182,...,-0.248402,-0.177659,0.000784,1.114785e-03,-0.971331,-0.237732,44.022409,33.840519,0,RandomSpeedOffset
1,1,3,52060.561117,1213.025174,984.277524,4.477449,4.459375,14.504808,2.605276,-0.008523,...,-0.653850,-0.117125,0.000001,1.847528e-07,0.992578,0.121610,2.560114,8.414909,0,DataReplay
2,2,3,28156.319142,140.514133,944.338854,2.965184,3.066191,-0.346027,4.671520,-0.000469,...,0.333247,-4.486889,0.000448,6.050771e-03,0.256103,0.966649,15.915074,9.636057,1,DoSDisruptive
3,3,3,28671.375689,558.005547,327.316562,4.934159,5.037039,11.792797,4.028876,0.022346,...,-0.030639,-0.010265,0.000066,2.237617e-05,0.954113,0.299446,2.854203,6.203941,1,RandomSpeedOffset
4,4,2,53612.000000,689.179631,547.143780,3.327547,3.374621,3.887137,-8.732709,0.000090,...,-1.829939,4.111129,0.000028,6.361425e-05,0.360402,-0.932797,5.648109,19.951521,0,DoS


In [7]:
print(data.shape)    # stampiamo il numero di righe e colonne
print(data.info())   # controlliamo i tipi di dato e valori nulli


(22165610, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22165610 entries, 0 to 22165609
Data columns (total 21 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Unnamed: 0   int64  
 1   type         int64  
 2   rcvTime      float64
 3   pos_0        float64
 4   pos_1        float64
 5   pos_noise_0  float64
 6   pos_noise_1  float64
 7   spd_0        float64
 8   spd_1        float64
 9   spd_noise_0  float64
 10  spd_noise_1  float64
 11  acl_0        float64
 12  acl_1        float64
 13  acl_noise_0  float64
 14  acl_noise_1  float64
 15  hed_0        float64
 16  hed_1        float64
 17  hed_noise_0  float64
 18  hed_noise_1  float64
 19  attack       int64  
 20  attack_type  object 
dtypes: float64(17), int64(3), object(1)
memory usage: 3.5+ GB
None


Possiamo notare come non ci siano valori mancanti (NaN)

In [8]:
data.info(verbose=True, show_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22165610 entries, 0 to 22165609
Data columns (total 21 columns):
 #   Column       Non-Null Count     Dtype  
---  ------       --------------     -----  
 0   Unnamed: 0   22165610 non-null  int64  
 1   type         22165610 non-null  int64  
 2   rcvTime      22165610 non-null  float64
 3   pos_0        22165610 non-null  float64
 4   pos_1        22165610 non-null  float64
 5   pos_noise_0  22165610 non-null  float64
 6   pos_noise_1  22165610 non-null  float64
 7   spd_0        22165610 non-null  float64
 8   spd_1        22165610 non-null  float64
 9   spd_noise_0  22165610 non-null  float64
 10  spd_noise_1  22165610 non-null  float64
 11  acl_0        22165610 non-null  float64
 12  acl_1        22165610 non-null  float64
 13  acl_noise_0  22165610 non-null  float64
 14  acl_noise_1  22165610 non-null  float64
 15  hed_0        22165610 non-null  float64
 16  hed_1        22165610 non-null  float64
 17  hed_noise_0  22165610 non

# Controllo e gestione dei duplicati

Osseriviamo che nel dataset non sono presenti righe duplicate e quindi non svolgiamo alcuna operazione di rimozione

In [9]:
# Conto quante righe duplicate ci sono
num_duplicati = data.duplicated().sum()
print(f"Numero di righe duplicate: {num_duplicati}")

# visualizzo le righe duplicate se > 0
if num_duplicati > 0:
    display(data[data.duplicated()].head())

Numero di righe duplicate: 0


# Analisi dei campi

In [10]:
# visualizzo i nomi di ciascun campo
for col in data.columns:
    print(col)


Unnamed: 0
type
rcvTime
pos_0
pos_1
pos_noise_0
pos_noise_1
spd_0
spd_1
spd_noise_0
spd_noise_1
acl_0
acl_1
acl_noise_0
acl_noise_1
hed_0
hed_1
hed_noise_0
hed_noise_1
attack
attack_type


Rimuovo campi irrilevanti (Unnamed) e tutte le colonne relative al rumore , informazioni non utilizzabili in un contesto reale. Mantengo invece le feature che includono position, speed, acceleration e heading del veicolo trasmittente che sono parametri a cui un sistema di rilevamento di misbehaviour detection ha accesso quando analizza i messaggi

In [11]:
# Lista delle colonne da rimuovere (nomi esatti dal dataset)
cols_to_remove = [
    'Unnamed: 0',
    'pos_noise_0', 'pos_noise_1',
    'spd_noise_0', 'spd_noise_1'
    'acl_noise_0', 'acl_noise_1',
    'hed_noise_0', 'hed_noise_1'
]

# Tiene solo quelle che esistono effettivamente in un contesto reale
cols_to_remove = [c for c in cols_to_remove if c in data.columns]

# Rimozione colonne
data = data.drop(columns=cols_to_remove)

print("Colonne rimanenti dopo la rimozione:")
print(data.columns)


Colonne rimanenti dopo la rimozione:
Index(['type', 'rcvTime', 'pos_0', 'pos_1', 'spd_0', 'spd_1', 'spd_noise_1',
       'acl_0', 'acl_1', 'acl_noise_0', 'hed_0', 'hed_1', 'attack',
       'attack_type'],
      dtype='object')


In [12]:
data = data.drop(columns=['spd_noise_1'], errors='ignore')
data = data.drop(columns=['acl_noise_0'], errors='ignore')
print(data.columns)

Index(['type', 'rcvTime', 'pos_0', 'pos_1', 'spd_0', 'spd_1', 'acl_0', 'acl_1',
       'hed_0', 'hed_1', 'attack', 'attack_type'],
      dtype='object')


il dataset contiene due tipologie di messaggi: i messaggi BSM e gli aggiornamenti GPS locali. Per rendere i dati più realistici e utilizzabili in uno scenario reale, gli aggiornamenti GPS locali (type = 2) vengono esclusi in quanto non sono messaggi ricevuti e non vengono utilizzati dai rilevatori di misbehaviour detection. Filtro il dataset per mantenere solo i messaggi BSM (type = 3)

In [13]:
# Filtro solo i messaggi ricevuti (type=3)
data = data[data['type'] == 3].copy()

# Ora posso rimuovere la colonna 'type' perché è tutta uguale
data = data.drop(columns=['type'])

print("Shape aggiornato:", data.shape)
print(data.head())

Shape aggiornato: (19112335, 11)
        rcvTime        pos_0       pos_1      spd_0     spd_1     acl_0  \
0  27371.216284   981.098535  908.497891 -17.723786 -2.154895 -0.248402   
1  52060.561117  1213.025174  984.277524  14.504808  2.605276 -0.653850   
2  28156.319142   140.514133  944.338854  -0.346027  4.671520  0.333247   
3  28671.375689   558.005547  327.316562  11.792797  4.028876 -0.030639   
5  26744.418540   262.104367  223.655554   8.191847  1.916282  1.406962   

      acl_1     hed_0     hed_1  attack        attack_type  
0 -0.177659 -0.971331 -0.237732       0  RandomSpeedOffset  
1 -0.117125  0.992578  0.121610       0         DataReplay  
2 -4.486889  0.256103  0.966649       1      DoSDisruptive  
3 -0.010265  0.954113  0.299446       1  RandomSpeedOffset  
5  0.329413  0.983696  0.179842       1          GridSybil  


In [14]:
data.head()

,rcvTime,pos_0,pos_1,spd_0,spd_1,acl_0,acl_1,hed_0,hed_1,attack,attack_type
0,27371.216284,981.098535,908.497891,-17.723786,-2.154895,-0.248402,-0.177659,-0.971331,-0.237732,0,RandomSpeedOffset
1,52060.561117,1213.025174,984.277524,14.504808,2.605276,-0.653850,-0.117125,0.992578,0.121610,0,DataReplay
2,28156.319142,140.514133,944.338854,-0.346027,4.671520,0.333247,-4.486889,0.256103,0.966649,1,DoSDisruptive
3,28671.375689,558.005547,327.316562,11.792797,4.028876,-0.030639,-0.010265,0.954113,0.299446,1,RandomSpeedOffset
5,26744.418540,262.104367,223.655554,8.191847,1.916282,1.406962,0.329413,0.983696,0.179842,1,GridSybil


# Identificazione degli attacchi presenti

Nel dataset fornito ho riscontrato la presenza di un numero di scenari di attacco maggiore rispetto ai cinque attacchi “default” citati nella documentazione originale del VeReMi. Nel dasaset originale vengono descritti solo attacchi di falsificazione della posizione: constpost, constposoffset, randompos, randomposoffset, eventualstop. Gli attacchi aggiuntivi nel dataset importato da kaggle possono derivare da scenari simulati extra e quindi ho filtrato il dataset mantenendo i 5 scenari di falsificazione della posizione relativi sia ai messaggi benigni con attack = 0 e malevoli con attack = 1

In [15]:
# Tutti i tipi unici di attacco nel dataset
attack_types = data['attack_type'].astype(str).str.strip().str.lower().unique()
print("Tipi di attacco presenti nel dataset:")
for att in attack_types:
    print("-", att)


Tipi di attacco presenti nel dataset:
- randomspeedoffset
- datareplay
- dosdisruptive
- gridsybil
- constposoffset
- datareplaysybil
- delayedmessages
- constspeedoffset
- dosdisruptivesybil
- randomposoffset
- eventualstop
- randomspeed
- constpos
- constspeed
- randompos
- dosrandom
- dos
- dosrandomsybil
- disruptive


filtro il dataset mantenendo i cinque scenari di attacco “default” definiti nel paper

In [16]:
keep = [
    "constpos",          # constant
    "constposoffset",    # constant offset
    "randompos",         # random
    "randomposoffset",   # random offset
    "eventualstop"       # eventual stop
]

# Normalizzo i nomi
data['attack_type'] = data['attack_type'].astype(str).str.strip().str.lower()

# Filtro solo i 5 attacchi, mantenendo attack=0 e attack=1
data_filtered = data[data['attack_type'].isin(keep)]

verifico l’effetto della selezione sui dati.

In [17]:
print("Righe prima della selezione:", len(data))
print("Righe dopo filtro:", len(data_filtered))
print("\nDistribuzione per tipo di attacco (comprende attack=0 e 1):")
print(data_filtered['attack_type'].value_counts())

Righe prima della selezione: 19112335
Righe dopo filtro: 4845815

Distribuzione per tipo di attacco (comprende attack=0 e 1):
attack_type
constposoffset     969163
randomposoffset    969163
eventualstop       969163
constpos           969163
randompos          969163
Name: count, dtype: int64


Dopo aver verificato il numero di righe prima e dopo il filtro e la distribuzione dei cinque attacchi di falsificazione della posizione, ho sovrascritto il DataFrame originale con la versione filtrata.


In [18]:
# dataset filtrato
data = data_filtered.reset_index(drop=True).copy()


In [19]:
# Controlli rapidi
print("Tipi unici dopo filtro:", sorted(data['attack_type'].unique()))
print("\nDistribuzione attack (0=benigno, 1=attacco):")
print(data['attack'].value_counts())

print("\nDistribuzione per attack_type x attack:")
print(pd.crosstab(data['attack_type'], data['attack']))

Tipi unici dopo filtro: ['constpos', 'constposoffset', 'eventualstop', 'randompos', 'randomposoffset']

Distribuzione attack (0=benigno, 1=attacco):
attack
0    2696310
1    2149505
Name: count, dtype: int64

Distribuzione per attack_type x attack:
attack                0       1
attack_type                    
constpos         539262  429901
constposoffset   539262  429901
eventualstop     539262  429901
randompos        539262  429901
randomposoffset  539262  429901


In [20]:
# controlli rapidi
data.shape
data.head()

,rcvTime,pos_0,pos_1,spd_0,spd_1,acl_0,acl_1,hed_0,hed_1,attack,attack_type
0,50831.914564,1288.736480,1059.729123,-16.287118,-2.921455,-0.055350,-0.009209,-0.998074,-0.062034,0,constposoffset
1,26663.698218,498.583238,303.468778,13.394928,4.750152,-0.562873,-0.199565,0.950863,0.309613,0,constposoffset
2,52161.314993,266.572055,41.134647,-0.531761,5.146397,-0.274390,2.655753,-0.095646,0.995415,0,constposoffset
3,27659.392320,365.106961,257.518958,14.903867,5.368904,-0.080198,-0.028698,0.954075,0.299568,1,randomposoffset
4,25623.210073,347.726613,483.105599,-9.744141,-7.657281,0.310752,0.244477,-0.799141,-0.601144,0,eventualstop
